In [32]:
import sys
!{sys.executable} -m pip install numpy pandas matplotlib scikit-learn seaborn imblearn 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

# import SMOTE and RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

#cross validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold

# logisitc regression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

#set seed

SEED = 618

You should consider upgrading via the 'c:\Users\ffinl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
# read in training data
train = pd.read_csv("/cs/studres/ID5059/Coursework/Coursework-2/data/train.csv")

# read in test data
test = pd.read_csv("/cs/studres/ID5059/Coursework/Coursework-2/data/test.csv")

# Clear the maximum number of columns to be displayed, so that all will be visible.
pd.set_option('display.max_columns', None)

display(train)
display(test)

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,0.0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,0.742144,0.108782,-1.070243,-0.234910,-1.099360,0.502467,0.169318,0.065688,-0.306957,-0.323800,0.103348,-0.292969,-0.334701,-0.887840,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0
1,1,0.0,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,-0.109948,0.908773,0.836798,-0.056580,-0.120990,-0.144028,-0.039582,1.653057,-0.253599,-0.814354,0.716784,0.065717,0.054848,-0.038367,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0
2,2,0.0,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,-0.042027,-0.818383,-0.376512,-0.226546,-0.552869,-0.886466,-0.180890,0.230286,0.590579,-0.321590,-0.433959,-0.021375,-0.326725,-0.803736,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0
3,3,0.0,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,0.885838,-0.007527,0.637441,0.676960,-1.504823,0.554039,-0.824356,-0.527267,-0.095838,-0.312519,0.642659,-0.340089,-0.095514,-0.079792,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0
4,4,0.0,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,0.692665,-0.097495,1.338869,1.391399,-0.128167,-0.081836,0.100548,-0.338937,0.090864,-0.423645,-0.731939,-0.203628,0.099157,0.608908,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219124,219124,120580.0,1.891079,-1.272908,-3.783908,-1.388939,2.012789,2.666080,0.151740,0.401934,-1.102824,0.858158,-0.280639,0.007976,-0.124950,0.914374,-0.073169,-2.309929,-0.041423,1.190526,-0.281848,-0.195703,-0.181369,-0.456538,-0.069571,0.756765,0.244479,-0.147566,-0.054725,-0.044588,198.65,0
219125,219125,120580.0,0.139724,0.948649,-2.913655,-2.184829,1.883716,-1.056824,1.725624,0.018089,-0.823494,-0.257933,-0.457534,0.516146,-0.071240,1.310799,-1.892909,-0.318780,-0.917395,0.098397,-0.195558,-0.116538,0.491469,1.478823,-0.085398,-0.091409,-1.053488,0.467570,0.358918,0.294735,24.00,0
219126,219126,120580.0,2.058343,-0.038993,-1.928553,0.330117,0.270127,-0.735664,-0.173878,0.144823,0.849289,-0.136498,0.179926,-1.769641,-3.937694,0.031346,0.028100,0.883566,0.391801,1.007789,0.303376,-0.384830,-0.306640,-0.965783,0.307799,-0.021434,-0.343989,0.181065,-0.098387,-0.044064,1.79,0
219127,219127,120580.0,2.079227,-2.162389,-1.785833,-2.804889,0.552845,4.038013,-2.155900,1.023785,-0.865242,1.536193,-0.058879,-0.885949,-0.254718,-0.425730,0.665556,-0.336634,0.301966,0.391249,0.037770,-0.190984,0.109909,0.590401,0.286621,0.675660,-0.510736,-0.090044,0.056749,-0.017126,88.00,0


,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,219129,120580.0,2.115519,-0.691809,-1.305514,-0.685655,-0.641265,-0.764784,-0.924262,-0.023030,-0.230126,0.220610,1.058325,-0.723363,-0.619241,-2.243711,-0.271429,2.007845,1.213534,0.282344,0.579121,0.067367,0.241708,0.682524,0.037769,-0.546859,-0.123055,-0.084889,0.004720,-0.021944,29.95
1,219130,120580.0,1.743525,-1.681429,-0.547387,-1.061113,-0.695825,2.458824,-1.632859,1.073529,1.068183,0.483337,0.274519,-0.346514,-0.409519,-0.564673,-0.143855,1.334855,0.160901,-0.087356,-0.066264,0.441788,0.543278,1.294571,0.309541,3.703925,-0.242579,0.068708,0.002629,0.064690,163.50
2,219131,120580.0,2.205568,-1.571445,-0.238965,-1.662517,-1.652324,-0.054701,-1.682064,0.105613,-1.177858,1.626352,0.601148,-0.041610,0.432748,-0.527684,-0.543665,0.043896,0.058472,0.622667,0.010671,-0.366906,-0.131527,0.086623,0.291375,0.739087,-0.543006,-0.297813,0.043699,-0.037855,16.00
3,219132,120580.0,1.989728,-0.972909,-1.938259,-1.440129,-0.166855,-0.794048,0.252889,-0.399789,2.079398,-1.225592,-0.869368,1.647638,1.138026,-0.258468,-0.361765,-1.253326,0.099479,-0.587702,1.207085,-0.049136,-0.080115,-0.010732,-0.038550,0.656830,0.343470,-0.627529,-0.024338,-0.036143,120.98
4,219133,120580.0,-1.943548,-1.668761,0.363601,-0.977610,2.684779,-2.037681,0.039709,-0.048895,-0.281749,-0.341879,0.471628,0.637604,-0.592416,0.331702,-1.451799,0.096467,-0.853439,-0.462060,-0.267098,0.391627,0.083389,-0.306918,0.247822,-0.391799,-0.790716,-0.025706,0.330758,0.335537,1.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146082,365211,172789.0,-0.661834,-0.622980,0.735580,-0.554067,0.869889,-0.389502,0.161226,0.069945,0.698829,-0.658935,-2.129855,-0.655069,-0.551589,-0.143536,-0.329767,0.625951,-0.949933,0.164734,0.079083,0.262408,-0.046068,-0.425692,0.253833,-1.076331,-0.999877,0.161938,0.149243,0.279708,94.81
146083,365212,172789.0,-0.098889,0.761900,0.340012,-0.589546,0.717091,-0.262765,0.726272,-0.070608,0.095958,-0.776078,-1.130049,0.506635,1.239831,-0.039782,0.446481,-0.049086,-0.746027,0.361708,-0.191256,0.064640,0.441215,1.284819,-0.141790,0.646930,-0.419964,-0.561426,0.326700,0.255166,24.99
146084,365213,172789.0,-1.487918,1.769142,0.551671,0.361566,0.056347,0.084844,0.567545,-0.059899,0.331706,1.623577,0.807877,1.021134,1.069455,-0.322923,0.063989,-0.089844,-0.764603,0.739254,0.291846,0.792736,0.163500,1.287079,-0.186955,-0.349847,-0.161724,-0.213933,0.392745,0.070938,29.99
146085,365214,172790.0,-1.889979,-0.154869,0.068496,-1.906964,-1.743181,0.717490,0.981608,-0.209713,-0.556087,-0.283025,-1.664145,-0.559516,1.238936,-0.887569,-1.155718,1.379725,0.114537,-1.190413,0.664771,-0.513829,0.121457,0.764642,-0.217844,0.104599,-0.008906,-0.204957,-0.752326,-0.155728,314.00


# Original Train Data

In [4]:
# set id as index
train = train.set_index('id')
test = test.set_index('id')

# split train set into X_train, y_train
y_train = train['Class']
X_train = train.drop('Class', axis = 1)

# SMOTE Balanced Data

In [5]:
# apply SMOTE to oversample the minority class
smote = SMOTE(sampling_strategy='minority')
X_train_SMOTE, y_train_SMOTE = smote.fit_resample(X_train, y_train)

# apply random undersampling to balance the class distribution
rus = RandomUnderSampler(sampling_strategy='majority')
X_train_SMOTE, y_train_SMOTE = rus.fit_resample(X_train_SMOTE, y_train_SMOTE)

# Decision Tree

In [12]:
# fit to original train with CV for parameter selection
# params = { 
#           'criterion':        ['gini', 'entropy', 'log_loss'], 
#          'min_samples_leaf': [75, 100] 
#         }

#tree_train = GridSearchCV(tree.DecisionTreeClassifier(), params, cv=2)
#tree_train = clf_train.fit(X_train, y_train)
#tree_train.best_estimator_

KeyboardInterrupt: 

In [ ]:
# fit to SMOTE train with CV for parameter selection
#params = { 
#          'criterion':        ['gini', 'entropy', 'log_loss'], 
#           'min_samples_leaf': [75, 100] 
#         }

#tree_SMOTE = GridSearchCV(tree.DecisionTreeClassifier(), params, cv=2)
#tree_SMOTE = clf_SMOTE.fit(X_train_SMOTE, y_train_SMOTE)
#tree_SMOTE.best_estimator_

In [6]:
#create tree object
tree1 = DecisionTreeClassifier(criterion = 'gini', min_samples_leaf = 100)
# train tree on original training data
tree_train = tree1.fit(X_train, y_train)

In [17]:
#create tree object
tree2 = DecisionTreeClassifier(criterion = 'gini', min_samples_leaf = 100)
# train tree on original training data
tree_SMOTE = tree2.fit(X_train, y_train)

## Cross Validation

In [6]:
# do k folds cross validation
num_folds = 5

# create stratfied k-fold object
strat_kfold = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42)

In [7]:
# do stratfied cv on train set 
tree_train_cv = cross_val_score(tree_train, X_train, y_train, cv=strat_kfold, scoring='roc_auc')

NameError: name 'tree_train' is not defined

In [43]:
# do stratified cv on SMOTE set 
tree_SMOTE_cv = cross_val_score(tree_SMOTE, X_train_SMOTE, y_train_SMOTE, cv=strat_kfold, scoring='roc_auc')

In [26]:
# compare cv scores
tree_meanCV = tree_train_cv

[0.67052682 0.6734344  0.64764421 0.61825765 0.6641468 ]
[0.98139506 0.97990637 0.98046229 0.98046783 0.98046803]


In [35]:
tree_train_cv_comp = pd.DataFrame[{'Original' = tree_meanCV, 'Imputed' = tree_meanCV_imputed}]


SyntaxError: cannot assign to literal here. Maybe you meant '==' instead of '='? (2395860588.py, line 1)

# Fit to Test

In [27]:
# fit trained tree
predicted_tree_train = tree_train.predict_proba(test)

In [28]:
# fit SMOTE trained tree
predicted_tree_SMOTE = tree_SMOTE.predict_proba(test)


## Export for Kaggle

In [7]:
# make a function to format results

def output_func(predictions):
    results = pd.DataFrame({'id': test.index, 'Class': predictions[:,1]})
    return results

In [40]:
tree_train_kaggle = output_func(predicted_tree_train)
tree_SMOTE_kaggle = output_func(predicted_tree_SMOTE)

In [8]:
# make function to export csv

def export_func (results, filename = 'out'):
    results.to_csv(filename, index = False)

In [42]:
# export csvs
export_func(tree_train_kaggle, 'tree_train_kaggle.csv')
export_func(tree_SMOTE_kaggle, 'tree_SMOTE_kaggle.csv')

train kaggle score: 0.68799
SMOTE kaggle score: 0.67115

# Logisitc Regression

In [9]:
# logisitic regression requires centered and scaled data

#create scaler objects for both types of data
scaler_train = StandardScaler()
scaler_SMOTE = StandardScaler()

# scale the training data
X_train_scaled = scaler_train.fit_transform(X_train)
X_SMOTE_scaled = scaler_SMOTE.fit_transform(X_train_SMOTE)

#scale the test data
X_test_scaled = scaler_train.transform(test)
X_test_SMOTE_scaled = scaler_SMOTE.transform(test)

In [11]:
# creat logisitic regression onject
logi_mod1 = LogisticRegression(random_state=42)
logi_mod2 = LogisticRegression(random_state=42)

In [12]:
# fit to both train and SMOTE
logi_train = logi_mod2.fit(X_train_scaled, y_train)
logi_SMOTE = logi_mod2.fit(X_SMOTE_scaled, y_train_SMOTE)

# Logisitc Regression using Eleastic Net

In [13]:
# set a parameter grid
# Define the hyperparameter grid
param_grid = {'l1_ratio': [0.5, 0.9]}
# define logisitc regressor with elastic net
logi_elastic = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=1000)

# Perform cross-validation grid search
grid_search = GridSearchCV(logi_elastic, param_grid, cv=4)
grid_search.fit(X_train, y_train)

# Choose the best hyperparameters
best_params = grid_search.best_params_

KeyboardInterrupt: 

In [ ]:
# Train the best model
logi_elastic_train = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000, **best_params)
logi_elastic_train.fit(X_train_ y_train)

## Cross Validation

In [27]:
# do stratified k fold on original train
logi_train_cv = cross_val_score(logi_train, X_train_scaled, y_train, cv = strat_kfold, scoring='roc_auc')

In [28]:
# do on smote data
logi_SMOTE_cv = cross_val_score(logi_SMOTE, X_SMOTE_scaled, y_train_SMOTE, cv = strat_kfold, scoring='roc_auc')

In [29]:
logi_train_meanCV = logi_train_cv.mean()
logi_SMOTE_meanCV = logi_SMOTE_cv.mean()

logi_train_meanCV
logi_SMOTE_meanCV

0.8675839649162649

In [ ]:
# make object for table
logi_train_cv_comp = pd.DataFrame['Original' = logi_train_meanCV, 'Imputed' = logi_train_imputed_meanCV]
logi_SMOTE_cv_comp = pd.DataFrame['Original' = logi_SMOTE_meanCV, 'Imputed' = logi_SMOTE_imputed_meanCV]
logi_elastic_cv_comp = pd.DataFrame['Original' = logi_elastic_meanCV, 'Imputed' = logi_elastic_imputed_meanCV]



# Fit to Test

In [60]:
# predict on test for both models
predicted_logi_train = logi_train.predict_proba(X_test_scaled)
predicted_logi_SMOTE = logi_SMOTE.predict_proba(X_test_SMOTE_scaled)

In [62]:
# create results array
logi_train_kaggle = output_func(predicted_logi_train)
logi_SMOTE_kaggle = output_func(predicted_logi_SMOTE)

In [63]:
# export csvs
export_func(logi_train_kaggle, 'logi_train_kaggle.csv')
export_func(logi_SMOTE_kaggle, 'logi_SMOTE_kaggle.csv')

In [ ]:
# kaggle scores
logi_train = 0.79312
logi_SMOTE = 0.81797


# Random Forrest

In [16]:
# Fit a Random Forrest Classifier with optimum paramters of the decision tree
rfc = RandomForestClassifier(n_estimators = 10, max_depth= 25, min_samples_leaf= 100, criterion= 'gini')

rfc_train = rfc.fit(X_train, y_train)

In [19]:
# do k folds cross validation
num_folds = 5

# create stratfied k-fold object
strat_kfold = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42)

# do stratified k fold on original train
rfc_train_cv = cross_val_score(rfc_train, X_train, y_train, cv = strat_kfold, scoring='roc_auc')

In [ ]:
# do k folds cross validation on imputed data set

In [25]:
rfc_meanCV = rfc_train_cv.mean()

0.7808581262511078

In [ ]:
# make object for table
rfc_cv_comp = pd.DataFrame['Original' = rfc_meanCV, 'Imputed' = rfc_meanCV_imputed]

In [23]:
rfc_predictions = rfc_train.predict_proba(test)
rfc_results = output_func(predictions= rfc_predictions)

In [24]:
export_func(rfc_results, filename= 'rfc_results.csv')

# ADA Boost

In [34]:
decision_tree_simple = DecisionTreeClassifier(max_depth = 1, random_state = SEED);
decision_tree_simple.fit(X_train, y_train)

ada_final = AdaBoostClassifier(decision_tree_simple, n_estimators = 250, algorithm = "SAMME.R", learning_rate = 0.1, random_state = SEED);
ada_final.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1,
                                                    random_state=618),
                   learning_rate=0.1, n_estimators=250, random_state=618)

In [ ]:
# perform cross validation original

num_folds = 5

# create stratfied k-fold object
strat_kfold = StratifiedKFold(n_splits = num_folds, shuffle=True, random_state=SEED)

# do stratified k fold on original train
ada_cv = cross_val_score(ada_final, X_train, y_train, cv = strat_kfold, scoring='roc_auc')

ada_meanCV = ada_cv.mean()

# perform cross validation on the imputed data

In [ ]:
# make object for table
ada_cv_comp = pd.DataFrame['Original' = ada_meanCV, 'Imputed' = ada_meanCV_imputed]

In [ ]:
# fit to test data and export for kaggle
ada_predictions = ada_final.predict_proba(test)

ada_results = output_func(ada_predictions)

export_func(ada_results, 'ada_results.csv')

# Results Table

In [75]:
# Create the first data frame
rfc_meanCV = [0.85]
rfc_meanCV_imputed = [0.84]
rfc_cv_comp = pd.DataFrame({'Original': rfc_meanCV, 'Imputed': rfc_meanCV_imputed})

# Create the second data frame
ada_meanCV = [0.81]
ada_meanCV_imputed = [0.79]
ada_cv_comp = pd.DataFrame({'Original': ada_meanCV, 'Imputed': ada_meanCV_imputed})

# Create a dictionary with the data frames and their names
dfs = {'tree'Random Forest': rfc_cv_comp, 'AdaBoost': ada_cv_comp}

# Concatenate the data frames and add the names as column labels
result = pd.concat([rfc_cv_comp, ada_cv_comp], axis = 0, keys=dfs.keys()).reset_index()
result

,level_0,level_1,Original,Imputed
0,Random Forest,0,0.85,0.84
1,AdaBoost,0,0.81,0.79


In [62]:
# Create a dictionary with the data frames and their names
dfs = {'Decision Tree': tree_train_cv_comp, 'Logistic Regression': logi_train_cv_comp, 
'SMOTE Logistic Regression': logi_SMOTE_cv_comp, 'Elastic Net Logistic Regression': logi_elastic_cv_comp,
 'Random Forest': rfc_cv_comp, 'AdaBoost': ada_cv_comp}

# Concatenate the data frames along the row axis
result = pd.concat(dfs.values(), axis=0, keys=dfs.keys())

# Print the resulting data frame
print(result)

         Random Forest AdaBoost
                     0        0
Original          0.85     0.81
Imputed           0.84     0.79
